In [20]:
import cv2 as cv
from ultralytics import YOLO
import os
import easyocr
import pandas as pd

In [10]:
vehicle_model=YOLO('yolov8n.pt')
liscence_model=YOLO('best.pt')

# Vehicle detection

In [6]:
# Initialize the OCR reader
reader = easyocr.Reader(['en'], gpu=True)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [15]:
# Mapping dictionaries for character conversion
# characters that can easily be confused can be 
# verified by their location - an `O` in a place
# where a number is expected is probably a `0`
dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}

dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}

In [16]:
def license_complies_format(text):
    # True if the license plate complies with the format, False otherwise.
    if len(text) != 7:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False

In [17]:
def format_license(text):
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_

In [18]:
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')

         # verify that text is conform to a standard license plate
        if license_complies_format(text):
            # bring text into the default license plate format
            return format_license(text), score

    return None, None

In [13]:
def write_csv(results, output_path):
    
    with open(output_path, 'w') as f:
        f.write('{},{},{},{},{},{},{},{}\n'.format(
            'frame_number', 'track_id', 'car_bbox', 'car_bbox_score',
            'license_plate_bbox', 'license_plate_bbox_score', 'license_plate_number',
            'license_text_score'))

        for frame_number in results.keys():
            for track_id in results[frame_number].keys():
                print(results[frame_number][track_id])
                if 'car' in results[frame_number][track_id].keys() and \
                   'license_plate' in results[frame_number][track_id].keys() and \
                   'number' in results[frame_number][track_id]['license_plate'].keys():
                    f.write('{},{},{},{},{},{},{},{}\n'.format(
                        frame_number,
                        track_id,
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['car']['bbox'][0],
                            results[frame_number][track_id]['car']['bbox'][1],
                            results[frame_number][track_id]['car']['bbox'][2],
                            results[frame_number][track_id]['car']['bbox'][3]
                        ),
                        results[frame_number][track_id]['car']['bbox_score'],
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['license_plate']['bbox'][0],
                            results[frame_number][track_id]['license_plate']['bbox'][1],
                            results[frame_number][track_id]['license_plate']['bbox'][2],
                            results[frame_number][track_id]['license_plate']['bbox'][3]
                        ),
                        results[frame_number][track_id]['license_plate']['bbox_score'],
                        results[frame_number][track_id]['license_plate']['number'],
                        results[frame_number][track_id]['license_plate']['text_score'])
                    )
        f.close()

In [14]:
vid="videos/vid1.mp4" 
video=cv.VideoCapture(vid)
results={}
ret = True
frame_number = -1
# all vehicle class IDs from the COCO dataset (car, motorbike, truck) https://docs.ultralytics.com/datasets/detect/coco/#dataset-yaml
vehicles = [2,3,5,7] #classes from COCO datasets

#NOTE: remove all files in 'plates' folder before running this script
#to be used only if you want to remove all files in 'plates' folder
# if os.path.exists('plates'):
#     for file in os.listdir('plates'):
#         os.remove(os.path.join('plates', file))
        
# get frame rate
frame_rate = 1

while ret:
    frame_number +=1
    ret, frame = video.read()
    # effectively skip frames
    video.set(cv.CAP_PROP_POS_MSEC, frame_number*1000*frame_rate)
    
    
    # break if no frame
    if not ret:
        break
    
    

    if ret and frame_number < 100:
        results[frame_number] = {}
    
        detections = vehicle_model.track(frame, persist=True, save=True)[0]
        for detection in detections.boxes.data.tolist():
            
            #This is to prevent errors when the model does not return class_id
            if len(detection) == 6:
                x1, y1, x2, y2, track_id, score = detection
                class_id = 0 # or any default value
            else:
                x1, y1, x2, y2, track_id, score, class_id = detection
            
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]

                    print(bbox)
                    
                    # license plate detector for region of interest
                    license_plates = liscence_model(roi)[0]
                    # print(license_plates)
                    # check every bounding box for a license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # verify detections
                        print(license_plate, 'track_id: ' + str(bbox[4]))
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # save license plate in folder called 'plates'
                        # displaying
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        np_text, np_score = read_license_plate(plate_treshold)
                            # if plate could be read write results
                        if np_text is not None:
                            results[frame_number][track_id] = {
                                'car': {
                                    'bbox': [x1, y1, x2, y2],
                                    'bbox_score': score
                                },
                                'license_plate': {
                                    'bbox': [plate_x1, plate_y1, plate_x2, plate_y2],
                                    'bbox_score': plate_score,
                                    'number': np_text,
                                    'text_score': np_score
                                }
                            }

                        if not os.path.exists('plates'):
                            os.makedirs('plates')
                        # cv.imwrite(str(f'plates/plate_{track_id}.jpg'), plate)
                        cv.imwrite(str(f'plates/plate_{track_id}' + '_gray.jpg'), plate_gray)
                        cv.imwrite(str(f'plates/plate_{track_id}' + '_thresh.jpg'), plate_treshold)


write_csv(results, './results.csv')
video.release()

0: 416x640 1 car, 526.4ms
Speed: 7.0ms preprocess, 526.4ms inference, 5.0ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs\detect\predict3
[97.86940002441406, 179.06077575683594, 996.3623046875, 644.4364624023438, 20.0, 0.8898076415061951]

0: 352x640 1 licence, 5341.2ms
Speed: 9.5ms preprocess, 5341.2ms inference, 2.0ms postprocess per image at shape (1, 3, 352, 640)
[504.7481689453125, 351.8430480957031, 683.5484008789062, 399.04736328125, 0.9147810339927673, 0.0] track_id: 20.0

0: 416x640 2 cars, 1205.1ms
Speed: 10.1ms preprocess, 1205.1ms inference, 11.8ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs\detect\predict3
[88.5030288696289, 215.37014770507812, 973.5576782226562, 681.8275146484375, 20.0, 0.8898076415061951]

0: 352x640 1 licence, 2078.0ms
Speed: 15.5ms preprocess, 2078.0ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)
[512.1406860351562, 315.4148864746094, 692.0692749023438, 364.8658447265625, 0.916204988

In [21]:
results = pd.read_csv('./results.csv')

# show results for tracking ID `1` - sort by OCR prediction confidence
results[results['track_id'] == 1.].sort_values(by='license_text_score', ascending=False)

,frame_number,track_id,car_bbox,car_bbox_score,license_plate_bbox,license_plate_bbox_score,license_plate_number,license_text_score
